# Train Last Layer WITH data augmentation via ImageDataGenerator object

The class ImageDataGenerator has objects, like `my_generator` below, can be used as the dataset in `model.fit` just like a tf.dataset can: instead of passing the training set and training labels, we simply pass the ImageDataGenerator object:

```python
model.fit(my_generator,...)
```
Here I follow <a href="https://stackoverflow.com/questions/56517963/keras-imagedatagenerator-for-segmentation-with-images-and-masks-in-separate-dire">a stackoverflow</a> post that seems to be slightly dated, since <a href="https://stepup.ai/train_data_augmentation_keras/">this blogpost</a>  says "**Note** that in previous releases of Keras, the function fit_generator() had to be used instead, but now fit() can handle both types of training!"

never mind... I use <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator">tf documentation examplefor segmentation masks</a>

## 1) sort filenames as usual

In [3]:
import numpy as np
from glob import glob
import os
import PIL
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
## retrieve model
# model = keras.models.load_model('saved_models/augdat_last_lay/')

## even a pre-trained model needs data:
# 1. Sort paths to files

IMAGE_SIZE = 224
path = 'beachlitter/'
images = sorted(glob(os.path.join(path, "images/*.jpg"))) # list of strings
masks = sorted(glob(os.path.join(path, "maskpngs/*.png"))) 

from sklearn.model_selection import train_test_split

def sort_paths(path, split=0.1):
    # PATH = "gs://dmcherney/beachlitter/"
    # glob(file_pattern) gives a list of strings satisfying the pattern
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "maskpngs/*")))

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)
    
    #shuffle with the same random seed to make sure masks stay with images.
    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = sort_paths(path)

### 1.1) Collect filenames into pandas dataframe

In [13]:
import pandas as pd

In [14]:
path_df = pd.DataFrame(columns=['train_x','train_y','fake_class'])
path_df['train_x'] = train_x
path_df['train_y'] = train_y
path_df['fake_class'] = np.ones(path_df['train_x'] .shape) 
path_df.head()

,train_x,train_y,fake_class
0,beachlitter/images/000711.jpg,beachlitter/maskpngs/000711.png,1.0
1,beachlitter/images/001911.jpg,beachlitter/maskpngs/001911.png,1.0
2,beachlitter/images/002911.jpg,beachlitter/maskpngs/002911.png,1.0
3,beachlitter/images/000652.jpg,beachlitter/maskpngs/000652.png,1.0
4,beachlitter/images/001688.jpg,beachlitter/maskpngs/001688.png,1.0


### 1.2) create two data generators, one for train images, one for train masks

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# make two image data generators with the same seed

SEED = 100
IMAGE_SIZE = 224
BATCH = 8
    
image_data_generator = ImageDataGenerator( # this has a train/validation split option I'm not using.
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    rotation_range = 10,
    zoom_range = 0.1
    ).flow_from_dataframe(dataframe= path_df,#Pandas dataframe containing the filepaths relative to directory (or absolute paths if directory is None
                          directory=None, # so, absolute
                          x_col='train_x', #string, column in dataframe that contains the absolute paths
                            y_col='fake_class', #string or list, column/s in dataframe that has the target data. 
                            weight_col=None,
                            target_size=(IMAGE_SIZE, IMAGE_SIZE),#default: (256, 256). The dimensions to which all images found will be resized.
                            color_mode='rgb', # one of "grayscale", "rgb", "rgba". Whether the images will be converted to have 1 or 3 color channels.
                            classes=None, # optional list of classes 
                            class_mode=None, #one of "binary", "categorical", "input", "multi_output", "raw", sparse" or None, last meaning generator will only yield batches of image data, which is useful to use in model.predict()
                            batch_size=BATCH,
                            shuffle=False, #default true, but I already did that, and I want the test set to match the other model fit runs.
                            seed=SEED, #for both shuffling and transformations, so I use a seed
                            save_to_dir=None, #optionally specify a directory to which to save the augmented pictures
                            save_prefix='',
                            save_format='png',
                            subset=None, # Subset of data ("training" or "validation") if validation_split is set in ImageDataGenerator.. might do that
                            interpolation='nearest', #Interpolation method used to resample the image if the target size is different from that of the loaded image
                            validate_filenames=True, #If True, invalid images will be ignored. Disabling this option can lead to speed-up
                        )

mask_data_generator = ImageDataGenerator(
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    rotation_range = 10,
    zoom_range = 0.1
    ).flow_from_dataframe(dataframe= path_df,#Pandas dataframe containing the filepaths relative to directory (or absolute paths if directory is None
                          directory=None, # so, absolute
                          x_col='train_y', #string, column in dataframe that contains the absolute paths
                            y_col='fake_class', #string or list, column/s in dataframe that has the target data. 
                            weight_col=None,
                            target_size=(IMAGE_SIZE, IMAGE_SIZE),#default: (256, 256). The dimensions to which all images found will be resized.
                            color_mode='rgb', # one of "grayscale", "rgb", "rgba". Whether the images will be converted to have 1 or 3 color channels.
                            classes=None, # optional list of classes 
                            class_mode=None, #one of "binary", "categorical", "input", "multi_output", "raw", sparse" or None, last meaning generator will only yield batches of image data, which is useful to use in model.predict()
                            batch_size=BATCH,
                            shuffle=False, #default true, but I already did that, and I want the test set to match the other model fit runs.
                            seed=SEED, #for both shuffling and transformations, so I use a seed
                            save_to_dir=None, #optionally specify a directory to which to save the augmented pictures
                            save_prefix='',
                            save_format='png',
                            subset=None, # Subset of data ("training" or "validation") if validation_split is set in ImageDataGenerator.. might do that
                            interpolation='nearest', #Interpolation method used to resample the image if the target size is different from that of the loaded image
                            validate_filenames=True, #If True, invalid images will be ignored. Disabling this option can lead to speed-up
                        )

Found 2800 validated image filenames.
Found 2800 validated image filenames.


In [28]:
mask_data_generator 

# how do I see if it is working/where the issue is? 

### 1.3) Combine generators into one

In [19]:
# This will put the image and maks data_generators together

def my_image_mask_generator(image_data_generator, mask_data_generator):
    train_generator = zip(image_data_generator, mask_data_generator)
    for (img, mask) in train_generator:
        yield (img, mask) # idk how this works

**Yield?**        

In [20]:
my_generator = my_image_mask_generator(image_data_generator, mask_data_generator)

## 2) test .fit the model with my_generator as data, with no val data form now.


In [21]:
# network architecture and fitting tools
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 
from tensorflow.keras.callbacks import TensorBoard 
# from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
import datetime


from tensorflow.keras.applications import MobileNetV2 

### 2.1) Define the model

In [22]:

def model():
    inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="input_image")
    
    # the pre-trained encoder
    encoder = MobileNetV2(input_tensor=inputs, 
                          weights="imagenet", # instead of randomized initial weights
                          include_top=False, # to fully connected softmax layer off. 
                          alpha=0.35) # proportionally decrease the number of filters in each layer. see paper.
    encoder.trainable = False
    skip_connection_names = ["input_image", "block_1_expand_relu", 
                             "block_3_expand_relu", "block_6_expand_relu"]
    encoder_output = encoder.get_layer("block_13_expand_relu").output
    # output of encoder is 16x16 
    
    # the decoder follows
    f = [16, 32, 48, 64] # the numbers of filters to use in skips traveling UP the U 
    x = encoder_output
    for i in range(1, len(skip_connection_names)+1, 1):
        x_skip = encoder.get_layer(skip_connection_names[-i]).output
        x = UpSampling2D((2, 2))(x)
        x = Concatenate()([x, x_skip])
        
        x = Conv2D(f[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
        x = Conv2D(f[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
    # output layer:
    # old version had one class, thus one filter:
    # x = Conv2D(1, (1, 1), padding="same")(x)
    # x = Activation("sigmoid")(x)
    x = Conv2D(8, (1, 1), padding="same")(x)
    x = Activation("softmax")(x)

    
    model = Model(inputs, x) # object created
    return model # object returned

model = model()
LR = 1e-4 #learning rate
opt = tf.keras.optimizers.Nadam(LR)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

### 2.2) Test of the .fit

In [27]:
# options first:
train_steps = len(train_x)//BATCH

#call fit
res = model.fit(
    my_generator, # the new guy... 
    epochs = 1,
    steps_per_epoch=train_steps, # cites error here, or whatever line is last, so I so not believe
)

InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
      self._run_once()
    File "/opt/

In [ ]:
1+1


In [ ]:
1+2

In [ ]:
# import pandas as pd
# scores= pd.DataFrame(data=res.history)
scores

In [ ]:
scores = pd.concat([scores,pd.DataFrame(data=res.history)])
scores

In [ ]:
scores.reset_index(inplace=True)

In [ ]:
model.save('saved_models/train_with_frozen_encoder/')

In [ ]:
plt.plot(scores['accuracy'], label='Train Accuracy')
plt.plot(scores['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title(f'Accuracy after {len(scores)} epochs')
plt.tight_layout();

In [ ]:
plt.plot(scores['loss'], label='Train Loss')
plt.plot(scores['val_loss'], label='Val Loss')
plt.legend()
plt.title(f'Accuracy after {len(scores)} epochs')
plt.tight_layout();

In [ ]:
%load_ext tensorboard


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
%%time
1+1